In [ ]:
import os
import torch
from torch import nn
import torch.optim 
import numpy as np  # to load dataset
import math

%matplotlib inline

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device in use: {device}")

## Load the dataset

In [ ]:
# define the dataset
# 0 corresponds to rotating
# 1 corresponds to grasping

# num rows for one grasp
block_size = 2 * 24

#linux command
# %cd /content/drive/MyDrive/PyTorch/

def normalize_vector(nd_arr: np.ndarray) -> np.ndarray:
    vector_length = sum([i ** 2 for i in nd_arr]) ** 0.5
    normalized_arr = [i / vector_length for i in nd_arr]
    return normalized_arr

# Rotation
unedited_rotating_dataset = np.loadtxt('rotating.csv', delimiter=",")
# Split into multiple movements over time ("block_size" seconds)
edited_rotating_dataset = unedited_rotating_dataset[:,1:21] #corresponding to cols 2 - 21 inclusive, excluding the last row of zeros
# Convert positions into velocities
edited_rotating_dataset = np.diff(edited_rotating_dataset, axis=0)
# Normalize each row
edited_rotating_dataset = np.array([normalize_vector(row) for row in edited_rotating_dataset])

# Grasping
unedited_grasping_dataset = np.loadtxt('grasping.csv', delimiter=",")
edited_grasping_dataset = unedited_grasping_dataset[:,1:21]
edited_grasping_dataset = np.diff(edited_grasping_dataset, axis=0)
# Normalize
edited_rotating_dataset = np.array([normalize_vector(row) for row in edited_grasping_dataset])

# Process data

In [ ]:
# Rotating
rows = edited_rotating_dataset.shape[0]
X = np.zeros((rows - block_size + 1, block_size * 20))  # (Block size * 24) seconds times 20 rows (from flattening)

# For each block
for i in range(0, rows - block_size + 1):
  # Get entire row, 1st block_size * 24 rows, 2nd of them and so on..
  X[i] = np.ndarray.flatten(edited_rotating_dataset[i : i + block_size, :])

# X now contains mutliple blocks
y = np.zeros((rows - block_size + 1))


# grasping
rows = edited_grasping_dataset.shape[0]
X2 = np.zeros((rows - block_size + 1, block_size * 20))
# For each block
for i in range(0, rows - block_size + 1):
  # Get entire row, 1st, block_size * 24 rows, second of them and so on..
  X2[i] = np.ndarray.flatten(edited_grasping_dataset[i : i + block_size, :])
# X now contains mutliple blocks
y2 = np.ones((rows - block_size + 1))
combined_X = np.concatenate((X, X2), axis=0)
combined_y = np.concatenate((y, y2), axis=0)

In [ ]:
# covnert into tensors since numpy uses 64 bit floating point
X = torch.tensor(combined_X, dtype=torch.float32)
y = torch.tensor(combined_y, dtype=torch.float32)
y = torch.reshape(y,(-1,1)) # a single dimension uses -1 to infer the length. new shape

print(X.shape, y.shape)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(block_size * 20, 64), # input
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork().to(device)
X.to(device)
y.to(device)
print(model)

## Preparation for training

In [ ]:
loss_fn = nn.BCELoss() # used for binary classification problems
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Training

In [ ]:
n_epochs = 15
batch_size = 4

# CPU
if device == "cpu":
  for epoch in range(n_epochs):
    for i in range(0, len(X) - batch_size, batch_size):
      Xbatch = X[i:i+batch_size] # specifying the rows
      y_pred = model(Xbatch) # scalar
      ybatch = y[i:i+batch_size]
      loss = loss_fn(y_pred, ybatch)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print(f"epoch: {epoch + 1}/{n_epochs}", end="\r")
  print(f'CPU: Finished epoch {epoch}, latest loss {loss}')

# GPU
elif device == "cuda":
  for epoch in range(n_epochs):
    for i in range(0, len(X) - batch_size, batch_size):
      Xbatch = X[i:i+batch_size] # specifying the rows
      y_pred = model(Xbatch.cuda()) # scalar
      ybatch = y[i:i+batch_size].cuda()
      loss = loss_fn(y_pred, ybatch)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print(f"epoch: {epoch + 1}/{n_epochs}", end="\r")
  print(f'CUDA: Finished epoch {epoch}, latest loss {loss}')
else:
  print("Not using CPU or CUDA")

In [ ]:
# evaluate the accuracy of the model

with torch.no_grad(): # no_grad to avoid differentiating
    X = X.to(device)
    y = y.to(device)
    y_pred = model(X.to(device))

# rounds prediction to nearest integer, check if equal to y, 
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")